In [137]:
import re
import pandas as pd
import demoji
from string import punctuation
import nltk
import ssl
import pickle

In [24]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/iamchetry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
d = pd.concat([pd.read_csv('dummy_csv_search_1.csv'), pd.read_csv('dummy_csv_search_2.csv'),
              pd.read_csv('dummy_csv_search_3.csv', lineterminator='\n'), pd.read_csv('dummy_csv_search_4.csv')],
              ignore_index=True, sort=True, axis=0)


In [3]:
d.drop(columns='user_name', axis=1, inplace=True)

In [4]:
df_poi = pd.read_csv('dummy_csv_poi.csv')
df_poi = df_poi[~df_poi.poi_name.isin(['@JoeBiden', '@POTUS'])]

df_poi_del = pd.read_csv('dummy_csv_poi_delta.csv')
df_poi = pd.concat([df_poi, df_poi_del], ignore_index=True, sort=True, axis=0)


In [5]:
d1 = pd.read_csv('dummy_replies_poi.csv')

In [6]:
d1.head(3)

,poi_name,poi_id,verified,country,id,replied_to_tweet_id,replied_to_user_id,reply_text,tweet_text,tweet_lang,hashtags,mentions,tweet_urls,tweet_date,geolocation
0,NaN,NaN,False,USA,1439962013703106561,NaN,1.447950e+09,"@AmitShah Sir pl ask @dir_ed, EOW, CBI etc to ...","@AmitShah Sir pl ask @dir_ed, EOW, CBI etc to ...",en,[],"['@AmitShah', '@dir_ed', '@DasShaktikanta', '@...",[],2021-09-20 14:37:51,NaN
1,NaN,NaN,False,INDIA,1439961865904263176,NaN,1.447950e+09,@AmitShah मैं जनवरी 2015 से सीoनंo 10324201715...,@AmitShah मैं जनवरी 2015 से सीoनंo 10324201715...,hi,[],['@AmitShah'],[],2021-09-20 14:37:16,NaN
2,NaN,NaN,False,INDIA,1439961427352035330,NaN,NaN,NaN,RT @SantKumarPande: @AmitShah @Bhupendrapbjp #...,hi,['#शिक्षक_ट्रांसफर_पोर्टल_चालू_करो'],"['@SantKumarPande', '@AmitShah', '@Bhupendrapb...",[],2021-09-20 14:35:32,NaN


In [7]:
d2 = pd.read_csv('dummy_replies_general.csv')

In [9]:
data_ = pd.concat([df_poi, d, d1, d2], ignore_index=True, sort=True, axis=0)
del df_poi, d, d1, d2

In [11]:
data_.drop(columns='geolocation', axis=1, inplace=True)

In [13]:
data_.head(1)

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,tweet_date,tweet_lang,tweet_text,tweet_urls,verified
0,USA,['#WeCanDoThis'],1440345427421786118,[],44783853.0,@HHSGov,NaN,NaN,NaN,2021-09-21 16:01:24,en,"With a COVID-19 vaccine, your favorite fall ac...",['http://vaccines.gov'],True


In [14]:
data_.hashtags = data_.hashtags.apply(eval)
data_.mentions = data_.mentions.apply(eval)
data_.tweet_urls = data_.tweet_urls.apply(eval)

data_.id = data_.id.astype('int').astype('str')

data_.poi_name = data_.poi_name.fillna('0')
data_.poi_name = data_.poi_name.astype('str')

data_.poi_id = data_.poi_id.fillna(0)
data_.poi_id = data_.poi_id.astype('int')

data_.replied_to_tweet_id = data_.replied_to_tweet_id.fillna(0)
data_.replied_to_tweet_id = data_.replied_to_tweet_id.astype('int')

data_.replied_to_user_id = data_.replied_to_user_id.fillna(0)
data_.replied_to_user_id = data_.replied_to_user_id.astype('int')

data_.reply_text = data_.reply_text.fillna('0')
data_.reply_text = data_.reply_text.astype('str')

data_.tweet_date = data_.tweet_date.apply(lambda _: pd.Timestamp(_).round('60min').to_pydatetime()).astype('str')

data_['tweet_text'] = data_.tweet_text.apply(lambda _: _.replace('RT ', ''))
data_['reply_text'] = data_.reply_text.apply(lambda _: _.replace('RT ', ''))


In [15]:
data_.head().to_dict(orient='records')

[{'country': 'USA',
  'hashtags': ['#WeCanDoThis'],
  'id': '1440345427421786118',
  'mentions': [],
  'poi_id': 44783853,
  'poi_name': '@HHSGov',
  'replied_to_tweet_id': 0,
  'replied_to_user_id': 0,
  'reply_text': '0',
  'tweet_date': '2021-09-21 16:00:00',
  'tweet_lang': 'en',
  'tweet_text': 'With a COVID-19 vaccine, your favorite fall activities are safer. \n\nFind vaccines near you at https://t.co/jDq2UIq3Yj. #WeCanDoThis https://t.co/lytXIxslvy',
  'tweet_urls': ['http://vaccines.gov'],
  'verified': True},
 {'country': 'USA',
  'hashtags': ['#COVID19', '#vaccine'],
  'id': '1440300231539625986',
  'mentions': ['@US_FDA'],
  'poi_id': 44783853,
  'poi_name': '@HHSGov',
  'replied_to_tweet_id': 0,
  'replied_to_user_id': 0,
  'reply_text': '0',
  'tweet_date': '2021-09-21 13:00:00',
  'tweet_lang': 'en',
  'tweet_text': 'Get your @US_FDA approved #COVID19 #vaccine today. Find a vaccination location near you at https://t.co/jDq2UIq3Yj. https://t.co/EJ0L2anFp3',
  'tweet_urls':

In [17]:
emoticons_happy = list([
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
    ])
emoticons_sad = list([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
    ])
all_emoticons = emoticons_happy + emoticons_sad

In [18]:
data_['tweet_emoticons'] = data_.tweet_text.apply(lambda _: list(demoji.findall(_).keys()))

In [25]:
stopword_en = nltk.corpus.stopwords.words('english')
stopword_es = nltk.corpus.stopwords.words('spanish')
stopword = stopword_en + stopword_es

In [28]:
def clean_text(text, lang):
    emojis = list(demoji.findall(text).keys())
    text = demoji.replace(text, '')
    
    if lang == 'hi':
        text = text.translate(str.maketrans('', '', punctuation))
        text = ' '.join([w for w in text.split() if not re.match(r'[A-Z0-9]+', w, re.I)])
    else:
        text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", text).split())
        text = ' '.join([_ for _ in text.split() if _.lower() not in stopword])

    return text

In [29]:
data_['cleaned_tweet'] = data_[['tweet_text', 'tweet_lang']].apply(lambda _: clean_text(_['tweet_text'], _['tweet_lang']), axis=1)


In [139]:
d = data_[data_.tweet_lang == 'hi']
d.index = range(len(d))

In [140]:
d

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,tweet_date,tweet_lang,tweet_text,tweet_urls,verified,tweet_emoticons,cleaned_tweet
0,INDIA,"[#Unite2FightCorona, #COVIDCharcha]",1440281542937354241,[@lavagarwal],2596143056,@MoHFW_INDIA,0,0,0,2021-09-21 12:00:00,hi,#Unite2FightCorona \n\nआने वाले त्योहारों में ...,[https://twitter.com/i/spaces/1ypJdgqQPWdGW],True,[],आने वाले त्योहारों में अनुरूप व्यवहार का पालन ...
1,INDIA,"[#Mumbai, #COVID19, #COVIDGroundZero, #Tyoharo...",1440259319899983885,"[@PMOIndia, @mansukhmandviya, @ianuragthakur]",2596143056,@MoHFW_INDIA,0,0,0,2021-09-21 10:00:00,hi,#Mumbai भर में बी.एम.सी द्वारा आयोजित महिला वि...,[],True,[],भर में बीएमसी द्वारा आयोजित महिला विशेष टीकाकर...
2,INDIA,[#COVID19],1440242289821962241,[],2596143056,@MoHFW_INDIA,1440241888062177280,2596143056,इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...,2021-09-21 09:00:00,hi,"""इस वीडियो में डॉ. अरविंद राजवंशी प्रोफेसर, एम...",[],True,[],इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...
3,INDIA,"[#IndiaFightsCorona, #COVID19]",1440241888062177284,"[@COVIDNewsByMIB, @DDNewslive, @airnewsalerts]",2596143056,@MoHFW_INDIA,1440241072760754176,2596143056,इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...,2021-09-21 09:00:00,hi,"#IndiaFightsCorona \n\n""इस वीडियो में डॉ. अरवि...",[],True,[],इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...
4,INDIA,"[#Unite2FightCorona, #COVID]",1440241072760754178,"[@PMOIndia, @mansukhmandviya, @ianuragthakur, ...",2596143056,@MoHFW_INDIA,0,0,0,2021-09-21 09:00:00,hi,"#Unite2FightCorona \n\nडॉ. अरविंद राजवंशी, प्र...",[],True,[],डॉ अरविंद राजवंशी प्रोफेसर एमडी कार्यकारी निदे...
5,INDIA,"[#WorldAlzheimersDay, #Alzheimer, #SwasthaBhar...",1440234678535868424,[],2596143056,@MoHFW_INDIA,1440234672760320000,2596143056,रोग के उपचार में पारिवारिक और सामाजिक सहयोग बह...,2021-09-21 09:00:00,hi,#WorldAlzheimersDay \n\n#Alzheimer's रोग के उप...,[],True,[],रोग के उपचार में पारिवारिक और सामाजिक सहयोग बह...
6,INDIA,"[#Alzheimer, #Dementia, #SwasthaBharat, #Healt...",1440191996312768516,[],2596143056,@MoHFW_INDIA,0,0,0,2021-09-21 06:00:00,hi,#Alzheimer's रोग और #Dementia से पीड़ित लोगों ...,[],True,[],रोग और से पीड़ित लोगों को अपना सहयोग दें।
7,INDIA,"[#LargestVaccineDrive, #Unite2FightCorona, #CO...",1440139674085847053,[],2596143056,@MoHFW_INDIA,1440139660521447424,2596143056,आज दोपहर से बजे जुड़ें डॉ अरविंद राजवंशी प्रोफ...,2021-09-21 02:00:00,hi,#LargestVaccineDrive #Unite2FightCorona \n\nआ...,[],True,[],आज दोपहर से बजे जुड़ें डॉ अरविंद राजवंशी प्रोफ...
8,INDIA,"[#HimachalPradesh, #COVID19, #COVIDGroundZero]",1439913995918450688,[@PMOIndia],2596143056,@MoHFW_INDIA,0,0,0,2021-09-20 11:00:00,hi,"मशोबरा, शिमला, #HimachalPradesh में निवासियों ...",[],True,[],मशोबरा शिमला में निवासियों को कोविड19 के टीके ...
9,INDIA,"[#Unite2FightCorona, #TyoharonKeRangCABKeSang,...",1439904000896212992,[@lavagarwal],2596143056,@MoHFW_INDIA,0,0,0,2021-09-20 11:00:00,hi,#Unite2FightCorona #TyoharonKeRangCABKeSang \n...,[https://twitter.com/i/spaces/1ypJdgqQPWdGW],True,[],ट्विटर के माध्यम से पर जुडें श्री संयुक्त सचिव...


In [35]:
for _ in range(500):
    print(d.cleaned_tweet[_])
    print('================================')

Juntos podemos parar COVID19 Quieres saber c mo Visita aprender c mo puedes unir lucha COVID 19 CombateCOVID JuntosPodemos
variante delta cambiado manera debemos protegernos COVID 19 Inf rmate nuevas recomendaciones CDC JuntosSiPodemosUSA
nete hoy 12 PM EST Regreso escuela preguntas vacunas COVID19 ser anfitri n reuni n responder preguntas compartir recursos educativos campa JuntoS Podemos HHS
ofrece informaci n espa ol trastornos salud mental temas relacionados pacientes familias profesionales salud p blico Explore nueva p gina NMMHM
Visite obtener recursos necesita espa ol ayudar compartir mensajes campa educaci n p blica COVID 19 HHS comunidad JuntosS PodemosUSA
sitio web espa ol campa educaci n p blica COVID 19 HHS est aqu recursos espa ol aumentar confianza p blico aceptaci n vacunas COVID 19 comunidad JuntosS PodemosUSA
sitio web espa ol campa educaci n p blica COVID 19 HHS est aqu recursos espa ol aumentar confianza p blico aceptaci n vacunas COVID 19 comunidad JuntosS PodemosUS

In [36]:
data_.head(1)

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,tweet_date,tweet_lang,tweet_text,tweet_urls,verified,tweet_emoticons,cleaned_tweet
0,USA,[#WeCanDoThis],1440345427421786118,[],44783853,@HHSGov,0,0,0,2021-09-21 16:00:00,en,"With a COVID-19 vaccine, your favorite fall ac...",[http://vaccines.gov],True,[],COVID 19 vaccine favorite fall activities safe...


In [41]:
def clean_replies(reply, cleaned):
    if reply == '0':
        return reply
    else:
        return cleaned

In [42]:
data_['reply_text'] = data_[['reply_text', 'cleaned_tweet']].apply(lambda _: clean_replies(_['reply_text'], _['cleaned_tweet']), axis=1)


In [43]:
data_[['reply_text', 'cleaned_tweet']]

,reply_text,cleaned_tweet
0,0,COVID 19 vaccine favorite fall activities safe...
1,0,Get FDA approved COVID19 vaccine today Find va...
2,0,awards 15M support expanding community based m...
3,0,Delta COVID 19 variants emerge important ever ...
4,0,Taking charge identity powerful take charge he...
5,0,health care provider collect information patie...
6,0,important ever get everyone vaccinated COVID 1...
7,Juntos podemos parar COVID19 Quieres saber c m...,Juntos podemos parar COVID19 Quieres saber c m...
8,0,Together stop COVID19 Want learn Learn availab...
9,0,Family precious Keep safe COVID 19 vaccine Fin...


In [45]:
len(data_), data_.id.nunique()

(122055, 120859)

In [50]:
d = data_.groupby(by='id').size().reset_index(name='counts_')

In [52]:
d.sort_values(by='counts_', ascending=False).head()

,id,counts_
77660,1440736768169365506,3
44637,1440241888062177284,3
104966,1440770357250510849,3
84290,1440747353527361541,2
80052,1440741020581318658,2


In [54]:
d = data_[data_.id.isin(['1440736768169365506', '1440241888062177284', '1440770357250510849'])]

In [56]:
d.sort_values(by='id', ascending=False)

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,tweet_date,tweet_lang,tweet_text,tweet_urls,verified,tweet_emoticons,cleaned_tweet
61500,USA,[],1440770357250510849,"[@GypsyGreed, @pierre_plourde, @SlanhofKelly, ...",0,0,1440708693536886784,32677526,plourde tx immune system fight Co,2021-09-22 20:00:00,en,@GypsyGreed @pierre_plourde @SlanhofKelly @for...,[https://twitter.com/i/web/status/144077035725...,False,[],plourde tx immune system fight Co
80699,USA,[],1440770357250510849,"[@GypsyGreed, @pierre_plourde, @SlanhofKelly, ...",0,0,1440708693536886784,32677526,plourde tx immune system fight Co,2021-09-22 20:00:00,en,@GypsyGreed @pierre_plourde @SlanhofKelly @for...,[https://twitter.com/i/web/status/144077035725...,False,[],plourde tx immune system fight Co
119634,USA,[],1440770357250510849,"[@GypsyGreed, @pierre_plourde, @SlanhofKelly, ...",0,0,1440708693536886784,32677526,plourde tx immune system fight Covid math prov...,2021-09-22 20:00:00,en,@GypsyGreed @pierre_plourde @SlanhofKelly @for...,[],False,[],plourde tx immune system fight Covid math prov...
49744,MEXICO,[],1440736768169365506,"[@aisdparents, @AustinCrockett, @AisdOffice, @...",0,0,1440736411766820864,63268295,OSL,2021-09-22 18:00:00,es,@aisdparents @AustinCrockett @AisdOffice @Aisd...,[https://twitter.com/i/web/status/144073676816...,True,[],OSL
88460,MEXICO,[],1440736768169365506,"[@aisdparents, @AustinCrockett, @AisdOffice, @...",0,0,1440736411766820864,63268295,OSL,2021-09-22 18:00:00,es,@aisdparents @AustinCrockett @AisdOffice @Aisd...,[https://twitter.com/i/web/status/144073676816...,True,[],OSL
113476,MEXICO,[],1440736768169365506,"[@aisdparents, @AustinCrockett, @AisdOffice, @...",0,0,1440736411766820864,63268295,OSL Colts oportunidad gratis ponerse vacuna Co...,2021-09-22 18:00:00,es,@aisdparents @AustinCrockett @AisdOffice @Aisd...,[https://www.austinisd.org/calendar/events/202...,True,[],OSL Colts oportunidad gratis ponerse vacuna Co...
6997,INDIA,"[#IndiaFightsCorona, #COVID19]",1440241888062177284,"[@COVIDNewsByMIB, @DDNewslive, @airnewsalerts]",2596143056,@MoHFW_INDIA,1440241072760754176,2596143056,इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...,2021-09-21 09:00:00,hi,"#IndiaFightsCorona \n\n""इस वीडियो में डॉ. अरवि...",[],True,[],इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...
44772,INDIA,"[#IndiaFightsCorona, #COVID19]",1440241888062177284,[],0,0,1440241072760754176,2596143056,इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...,2021-09-21 09:00:00,hi,"#IndiaFightsCorona \n\n""इस वीडियो में डॉ. अरवि...",[https://twitter.com/i/web/status/144024188806...,True,[],इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...
101809,INDIA,"[#IndiaFightsCorona, #COVID19]",1440241888062177284,"[@COVIDNewsByMIB, @DDNewslive, @airnewsalerts]",0,0,1440241072760754176,2596143056,इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...,2021-09-21 09:00:00,hi,"#IndiaFightsCorona \n\n""इस वीडियो में डॉ. अरवि...",[],True,[],इस वीडियो में डॉ अरविंद राजवंशी प्रोफेसर एमडी ...


In [62]:
data_.poi_name.nunique()

26

In [64]:
data_.groupby(by='poi_name').id.nunique()


poi_name
0                  91002
@AmitShah           1001
@ArvindKejriwal     1001
@BarackObama        1001
@CDCgov             1001
@EPN                2922
@HHSGov             1001
@HillaryClinton     1001
@JoeBiden            819
@KamalaHarris       1001
@MamataOfficial     1001
@MoHFW_INDIA         928
@PMOIndia           1001
@POTUS              1001
@RahulGandhi        1001
@RicardoAnayaC      2641
@SSalud_mx          1001
@ShashiTharoor      1001
@VP                 1001
@VP45                984
@drharshvardhan     1001
@lopezobrador_      1001
@marcelotorresc     1950
@mario_delgado      1652
@narendramodi       1001
@rajnathsingh       1001
Name: id, dtype: int64

In [68]:
data_[data_.reply_text != '0'].id.nunique()

30542

In [69]:
pois = list(data_[data_.poi_id != 0].poi_id.unique())

In [121]:
d = data_[~data_.replied_to_user_id.isin(pois)]

In [122]:
d.id.nunique()

13859

In [77]:
data_.head().to_dict(orient='records')

[{'country': 'USA',
  'hashtags': ['#WeCanDoThis'],
  'id': '1440345427421786118',
  'mentions': [],
  'poi_id': 44783853,
  'poi_name': '@HHSGov',
  'replied_to_tweet_id': 0,
  'replied_to_user_id': 0,
  'reply_text': '0',
  'tweet_date': '2021-09-21 16:00:00',
  'tweet_lang': 'en',
  'tweet_text': 'With a COVID-19 vaccine, your favorite fall activities are safer. \n\nFind vaccines near you at https://t.co/jDq2UIq3Yj. #WeCanDoThis https://t.co/lytXIxslvy',
  'tweet_urls': ['http://vaccines.gov'],
  'verified': True,
  'tweet_emoticons': [],
  'cleaned_tweet': 'COVID 19 vaccine favorite fall activities safer Find vaccines near WeCanDoThis'},
 {'country': 'USA',
  'hashtags': ['#COVID19', '#vaccine'],
  'id': '1440300231539625986',
  'mentions': ['@US_FDA'],
  'poi_id': 44783853,
  'poi_name': '@HHSGov',
  'replied_to_tweet_id': 0,
  'replied_to_user_id': 0,
  'reply_text': '0',
  'tweet_date': '2021-09-21 13:00:00',
  'tweet_lang': 'en',
  'tweet_text': 'Get your @US_FDA approved #COVI

In [78]:
data_.memory_usage().sum()

14768735

In [96]:
data_.head(1)

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,tweet_date,tweet_lang,tweet_text,tweet_urls,verified,tweet_emoticons,cleaned_tweet
0,USA,[#WeCanDoThis],1440345427421786118,[],44783853,@HHSGov,0,0,0,2021-09-21 16:00:00,en,"With a COVID-19 vaccine, your favorite fall ac...",[http://vaccines.gov],True,[],COVID 19 vaccine favorite fall activities safe...


In [97]:
collection = data_.to_dict(orient='records')

In [104]:
collection[0]

{'country': 'USA',
 'hashtags': ['#WeCanDoThis'],
 'id': '1440345427421786118',
 'poi_id': 44783853,
 'poi_name': '@HHSGov',
 'replied_to_tweet_id': 0,
 'replied_to_user_id': 0,
 'reply_text': '0',
 'tweet_date': '2021-09-21 16:00:00',
 'tweet_lang': 'en',
 'tweet_text': 'With a COVID-19 vaccine, your favorite fall activities are safer. \n\nFind vaccines near you at https://t.co/jDq2UIq3Yj. #WeCanDoThis https://t.co/lytXIxslvy',
 'tweet_urls': ['http://vaccines.gov'],
 'verified': True,
 'text_en': 'COVID 19 vaccine favorite fall activities safer Find vaccines near WeCanDoThis'}

In [99]:
for dict_ in collection:
    dict_['text_'+dict_['tweet_lang']] = dict_['cleaned_tweet']

In [101]:
for dict_ in collection:
    del dict_['cleaned_tweet']

In [103]:
for dict_ in collection:
    for _ in list(dict_.keys()):
        if isinstance(dict_[_], list) and not dict_[_]:
            del dict_[_]
            

In [105]:
for dict_ in collection:
    if dict_['replied_to_tweet_id'] == 0 or dict_['replied_to_user_id'] == 0 or dict_['reply_text'] == '0':
        del dict_['replied_to_tweet_id']
        del dict_['replied_to_user_id']
        del dict_['reply_text']
        

In [106]:
collection[0]

{'country': 'USA',
 'hashtags': ['#WeCanDoThis'],
 'id': '1440345427421786118',
 'poi_id': 44783853,
 'poi_name': '@HHSGov',
 'tweet_date': '2021-09-21 16:00:00',
 'tweet_lang': 'en',
 'tweet_text': 'With a COVID-19 vaccine, your favorite fall activities are safer. \n\nFind vaccines near you at https://t.co/jDq2UIq3Yj. #WeCanDoThis https://t.co/lytXIxslvy',
 'tweet_urls': ['http://vaccines.gov'],
 'verified': True,
 'text_en': 'COVID 19 vaccine favorite fall activities safer Find vaccines near WeCanDoThis'}

In [107]:
df_new = pd.DataFrame(collection)

In [112]:
df_new.head(2)

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,text_en,text_es,text_hi,tweet_date,tweet_emoticons,tweet_lang,tweet_text,tweet_urls,verified
0,USA,[#WeCanDoThis],1440345427421786118,NaN,44783853,@HHSGov,NaN,NaN,NaN,COVID 19 vaccine favorite fall activities safe...,NaN,NaN,2021-09-21 16:00:00,NaN,en,"With a COVID-19 vaccine, your favorite fall ac...",[http://vaccines.gov],True
1,USA,"[#COVID19, #vaccine]",1440300231539625986,[@US_FDA],44783853,@HHSGov,NaN,NaN,NaN,Get FDA approved COVID19 vaccine today Find va...,NaN,NaN,2021-09-21 13:00:00,NaN,en,Get your @US_FDA approved #COVID19 #vaccine to...,[http://vaccines.gov],True


In [114]:
df_new.id = df_new.id.astype('int').astype('str')

df_new.poi_name = df_new.poi_name.fillna('0')
df_new.poi_name = df_new.poi_name.astype('str')

df_new.poi_id = df_new.poi_id.fillna(0)
df_new.poi_id = df_new.poi_id.astype('int')

df_new.replied_to_tweet_id = df_new.replied_to_tweet_id.fillna(0)
df_new.replied_to_tweet_id = df_new.replied_to_tweet_id.astype('int')

df_new.replied_to_user_id = df_new.replied_to_user_id.fillna(0)
df_new.replied_to_user_id = df_new.replied_to_user_id.astype('int')

df_new.reply_text = df_new.reply_text.fillna('0')
df_new.reply_text = df_new.reply_text.astype('str')


In [115]:
df_new.head(1)

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,text_en,text_es,text_hi,tweet_date,tweet_emoticons,tweet_lang,tweet_text,tweet_urls,verified
0,USA,[#WeCanDoThis],1440345427421786118,NaN,44783853,@HHSGov,0,0,0,COVID 19 vaccine favorite fall activities safe...,NaN,NaN,2021-09-21 16:00:00,NaN,en,"With a COVID-19 vaccine, your favorite fall ac...",[http://vaccines.gov],True


In [129]:
df_new[df_new.reply_text != '0'][df_new.replied_to_user_id.isin(pois)].id.nunique()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


10223

In [128]:
df_new[df_new.replied_to_user_id.isin(pois)].replied_to_tweet_id.nunique()

2243

In [132]:
for dict_ in collection:
    if dict_['poi_name'] == '0':
        del dict_['poi_name']
        del dict_['poi_id']

In [135]:
for dict_ in collection:
    if 'tweet_emoticons' in dict_:
        print(dict_)
        break

{'country': 'USA', 'hashtags': ['#FluShot', '#flu', '#vaccines'], 'id': '1439212617705340928', 'poi_id': 44783853, 'poi_name': '@HHSGov', 'tweet_date': '2021-09-18 13:00:00', 'tweet_lang': 'en', 'tweet_text': 'Getting a #FluShot \U0001fa79 is an easy way to protect yourself and others from the flu this year. Learn more about #flu #vaccines and how they work: https://t.co/ukWnbUbIBc. https://t.co/jsfcrQxZa5', 'tweet_urls': ['https://go.usa.gov/xMZWk'], 'verified': True, 'tweet_emoticons': ['\U0001fa79'], 'text_en': 'Getting FluShot easy way protect others flu year Learn flu vaccines work'}


In [136]:
len(collection)

122055

In [138]:
with open('docs.pickle', 'wb') as f:
    pickle.dump(collection, f, protocol=pickle.HIGHEST_PROTOCOL)

In [141]:
with open('crowdsourced_keywords.pickle', 'rb') as f:
    dict_ = pickle.load(f)

In [151]:
dict_

{'covid': ['quarentena',
  'hospital',
  'covidresources',
  'rt-pcr',
  'वैश्विकमहामारी',
  'oxygen',
  'सुरक्षित रहें',
  'stayhomestaysafe',
  'covid19',
  'quarantine',
  'मास्क',
  'face mask',
  'covidsecondwaveinindia',
  'flattenthecurve',
  'corona virus',
  'wuhan',
  'cierredeemergencia',
  'autoaislamiento',
  'sintomas',
  'covid positive',
  'casos',
  'कोविड मृत्यु',
  'स्वयं चुना एकांत',
  'stay safe',
  '#deltavariant',
  'covid symptoms',
  'sarscov2',
  'covidiots',
  'brote',
  'alcohol en gel',
  'disease',
  'asintomático',
  'टीकाकरण',
  'encierro',
  'covidiot',
  'covidappropriatebehaviour',
  'fever',
  'pandemia de covid-19',
  'wearamask',
  'flatten the curve',
  'oxígeno',
  'desinfectante',
  'super-spreader',
  'ventilador',
  'coronawarriors',
  'quedate en casa',
  'mascaras',
  'mascara facial',
  'trabajar desde casa',
  'संगरोध',
  'immunity',
  'स्वयं संगरोध',
  'डेल्टा संस्करण',
  'mask mandate',
  'health',
  'dogajkidoori',
  'travelban',
  'cil

In [181]:
covid_kws = ["quarentena",
  "hospital",
  "covidresources",
  "rt-pcr",
  "वैश्विकमहामारी",
  "oxygen",
  "सुरक्षित रहें",
  "stayhomestaysafe",
  "covid19",
  "quarantine",
  "मास्क",
  "face mask",
  "covidsecondwaveinindia",
  "flattenthecurve",
  "corona virus",
  "wuhan",
  "cierredeemergencia",
  "autoaislamiento",
  "sintomas",
  "covid positive",
  "casos",
  "कोविड मृत्यु",
  "स्वयं चुना एकांत",
  "stay safe",
  "#deltavariant",
  "covid symptoms",
  "sarscov2",
  "covidiots",
  "brote",
  "alcohol en gel",
  "disease",
  "asintomático",
  "टीकाकरण",
  "encierro",
  "covidiot",
  "covidappropriatebehaviour",
  "fever",
  "pandemia de covid-19",
  "wearamask",
  "flatten the curve",
  "oxígeno",
  "desinfectante",
  "super-spreader",
  "ventilador",
  "coronawarriors",
  "quedate en casa",
  "mascaras",
  "mascara facial",
  "trabajar desde casa",
  "संगरोध",
  "immunity",
  "स्वयं संगरोध",
  "डेल्टा संस्करण",
  "mask mandate",
  "health",
  "dogajkidoori",
  "travelban",
  "cilindro de oxígeno",
  "covid",
  "staysafe",
  "variant",
  "yomequedoencasa",
  "doctor",
  "एंटीबॉडी",
  "दूसरी लहर",
  "distancia social",
  "मुखौटा",
  "covid test",
  "अस्पताल",
  "covid deaths",
  "कोविड19",
  "muvariant",
  "susanadistancia",
  "personal protective equipment",
  "remdisivir",
  "quedateencasa",
  "asymptomatic",
  "social distancing",
  "distanciamiento social",
  "cdc",
  "transmission",
  "epidemic",
  "social distance",
  "herd immunity",
  "transmisión",
  "सैनिटाइज़र",
  "indiafightscorona",
  "surgical mask",
  "facemask",
  "desinfectar",
  "वायरस",
  "संक्रमण",
  "symptoms",
  "सामाजिक दूरी",
  "covid cases",
  "ppe",
  "sars",
  "autocuarentena",
  "प्रक्षालक",
  "breakthechain",
  "stayhomesavelives",
  "coronavirusupdates",
  "sanitize",
  "covidinquirynow",
  "कोरोना",
  "workfromhome",
  "outbreak",
  "flu",
  "sanitizer",
  "distanciamientosocial",
  "variante",
  "कोविड 19",
  "कोविड-19",
  "covid pneumonia",
  "कोविड",
  "pandemic",
  "icu",
  "वाइरस",
  "contagios",
  "वेंटिलेटर",
  "washyourhands",
  "n95",
  "stayhome",
  "lavadodemanos",
  "fauci",
  "रोग प्रतिरोधक शक्ति",
  "maskmandate",
  "डेल्टा",
  "कोविड महामारी",
  "third wave",
  "epidemia",
  "fiebre",
  "मौत",
  "travel ban",
  "फ़्लू",
  "muerte",
  "स्वच्छ",
  "washhands",
  "enfermedad",
  "contagio",
  "infección",
  "faceshield",
  "self-quarantine",
  "remdesivir",
  "oxygen cylinder",
  "mypandemicsurvivalplan",
  "कोविड के केस",
  "delta variant",
  "wuhan virus",
  "लक्षण",
  "corona",
  "maskup",
  "gocoronago",
  "death",
  "curfew",
  "socialdistance",
  "second wave",
  "máscara",
  "stayathome",
  "positive",
  "lockdown",
  "propagación en la comunidad",
  "तीसरी लहर",
  "aislamiento",
  "rtpcr",
  "coronavirus",
  "variante delta",
  "distanciasocial",
  "cubrebocas",
  "घर पर रहें",
  "socialdistancing",
  "covidwarriors",
  "प्रकोप",
  "covid-19",
  "stay home",
  "संक्रमित",
  "jantacurfew",
  "cowin",
  "कोरोनावाइरस",
  "virus",
  "distanciamiento",
  "cuarentena",
  "indiafightscovid19",
  "healthcare",
  "natocorona",
  "मास्क पहनें",
  "delta",
  "ऑक्सीजन",
  "wearmask",
  "कोरोनावायरस",
  "ventilator",
  "pneumonia",
  "maskupindia",
  "ppe kit",
  "sars-cov-2",
  "testing",
  "fightagainstcovid19",
  "महामारी",
  "नियंत्रण क्षेत्र",
  "who",
  "mask",
  "pandemia",
  "deltavariant",
  "वैश्विक महामारी",
  "रोग",
  "síntomas",
  "work from home",
  "antibodies",
  "masks",
  "confinamiento",
  "flattening the curve",
  "मुखौटा जनादेश",
  "thirdwave",
  "mascarilla",
  "usacubrebocas",
  "covidemergency",
  "inmunidad",
  "cierre de emergencia",
  "self-isolation",
  "स्वास्थ्य सेवा",
  "सोशल डिस्टन्सिंग",
  "isolation",
  "cases",
  "community spread",
  "unite2fightcorona",
  "oxygencrisis",
  "containment zones",
  "homequarantine",
  "स्पर्शोन्मुख",
  "लॉकडाउन",
  "hospitalización",
  "incubation period"]

In [164]:
vaccine_kwds = ["anticuerpos",
  "vaccine mandate",
  "eficacia de la vacuna",
  "vacuna covid",
  "covidvaccine",
  "zycov-d",
  "vaccines",
  "#largestvaccinedrive",
  "vaccination",
  "dosis de vacuna",
  "moderna",
  "campaña de vacunación",
  "vaccineshortage",
  "vacunar",
  "covid vaccine",
  "efectos secundarios de la vacuna",
  "कोविशील्ड",
  "hydroxychloroquine",
  "efficacy",
  "टीके",
  "टीकाकरण",
  "वैक्सीनेशन",
  "shots",
  "covishield",
  "vaccine",
  "antibody",
  "j&j vaccine",
  "booster shot",
  "वैक्सीन पासपोर्ट",
  "covidvaccination",
  "दूसरी खुराक",
  "inyección de refuerzo",
  "astrazeneca",
  "टीकाकरण अभियान",
  "vacunacovid19",
  "johnson & johnson",
  "पहली खुराक",
  "sinopharm",
  "immunity",
  "vaccination drive",
  "inmunización",
  "vaccine dose",
  "we4vaccine",
  "पूर्ण टीकाकरण",
  "vaccine passports",
  "एंटीबॉडी",
  "vacunado",
  "vacunarse",
  "johnson",
  "efecto secundario",
  "astra zeneca",
  "yomevacunoseguro",
  "injection",
  "cdc",
  "वैक्सीन के साइड इफेक्ट",
  "getvaxxed",
  "teeka",
  "टीका",
  "herd immunity",
  "वैक्सीन जनादेश",
  "vaccinepassports",
  "estrategiadevacunación",
  "ivermectin",
  "cansino",
  "vacunas",
  "vaccinehesitancy",
  "sputnik",
  "johnson & johnson’s janssen",
  "unvaccinated",
  "janssen",
  "sputnik v",
  "vacunaton",
  "seconddose",
  "कोवेक्सिन",
  "getvaccinatednow",
  "tikakaran",
  "कोविशिल्ड",
  "खुराक",
  "covaxine",
  "mrna",
  "first dose",
  "वाइरस",
  "booster shots",
  "dosis",
  "side effect",
  "रोग प्रतिरोधक शक्ति",
  "jab",
  "get vaccinated",
  "vaccinessavelives",
  "pinchazo",
  "vaccinesideeffects",
  "vaccinated",
  "कोविड का टीका",
  "खराब असर",
  "vacunación",
  "कोवैक्सिन",
  "tikautsav",
  "efectos secundarios",
  "remdesivir",
  "covid19vaccine",
  "eficacia",
  "anticuerpo",
  "vaccinequity",
  "vaccinesamvaad",
  "फाइजर",
  "vaccinesamvad",
  "covid-19 vaccine",
  "pasaporte de vacuna",
  "largestvaccinationdrive",
  "firstdose",
  "doses",
  "vacuna",
  "la inmunidad de grupo",
  "कोवैक्सीन",
  "vaccine side effects",
  "कोविन",
  "vaccinationdrive",
  "clinical trial",
  "vaccinemandate",
  "segunda dosis",
  "cowin",
  "vaccinate",
  "clinical trials",
  "fully vaccinated",
  "johnson and johnson",
  "primera dosis",
  "largestvaccinedrive",
  "vaccine hesitancy",
  "वैक्सीन",
  "प्रभाव",
  "vacunacion",
  "second dose",
  "sabkovaccinemuftvaccine",
  "लसीकरण",
  "vaccineswork",
  "वैक्\u200dसीन",
  "दुष्प्रभाव",
  "pfizer",
  "vaccine efficacy",
  "टीका लगवाएं",
  "एमआरएनए वैक्सीन",
  "antibodies",
  "getvaccinated",
  "covidshield",
  "booster",
  "टीका_जीत_का",
  "vaccine jab",
  "vaccine passport",
  "vaccinepassport",
  "mrna vaccine",
  "inmunidad",
  "एस्ट्राजेनेका",
  "mandato de vacuna",
  "astrazenca",
  "vacúnate",
  "vacuna para el covid-19",
  "vacunada",
  "side effects",
  "dose",
  "novavax",
  "j&j",
  "covaxin",
  "fullyvaccinated",
  "sputnikv",
  "कोविड टीका",
  "completamente vacunado",
  "novaccinepassports",
  "sinovac"]

In [3]:
d.id = d.id.astype('int')

In [6]:
d_count = d.groupby(by='user_name').id.nunique().reset_index()

In [8]:
mult = list(d_count[d_count.id > 1].user_name)

In [12]:
d_count[d_count.user_name.isin(mult)]

,user_name,id
0,@0001Cecilia,3
4,@0071061Mukesh,2
7,@00CarlosCabrera,2
10,@01kris,4
21,@07VinodYadav,3
31,@0RobertoSanchez,2
32,@0bFuSc8,4
48,@101DSV,2
53,@1027phoenix,2
55,@103FMTrinidad,2


In [13]:
data_ = d[d.user_name.isin(mult)]

In [15]:
data_.user_name.nunique()

7293

In [16]:
df_grouped = data_.groupby(by='user_name').agg({'id': ['min', 'max']}).reset_index()

In [19]:
df_grouped = pd.concat([df_grouped.user_name, df_grouped.id.rename(columns={'min': 'min_id', 'max': 'max_id'})], ignore_index=False, axis=1)


In [23]:
df_grouped.to_csv('df_general_reply_raw.csv', index=False)

In [22]:
df_grouped

,user_name,min_id,max_id
0,@0001Cecilia,1440785071561064449,1440786056580788227
1,@0071061Mukesh,1438462615609483264,1440710763065208832
2,@00CarlosCabrera,1440746723811282956,1440749888560504837
3,@01kris,1440766565511696389,1440769807733780486
4,@07VinodYadav,1438192604529639424,1440279062388174860
5,@0RobertoSanchez,1440720906003750914,1440743579689766916
6,@0bFuSc8,1440752934073950211,1440792985252208642
7,@101DSV,1438352531017457664,1440683873227079683
8,@1027phoenix,1440756417749610500,1440759734286512136
9,@103FMTrinidad,1440749021929164801,1440762007012065289


In [8]:
df_poi.drop_duplicates(subset=['id'], inplace=True)

In [10]:
df_poi['tweet_text'] = df_poi.tweet_text.apply(lambda _: _.replace('RT ', ''))

In [12]:
df_poi.id = df_poi.id.astype('int')

In [15]:
df_grouped = df_poi.groupby(by='poi_name').agg({'id': ['min', 'max']}).reset_index()

In [17]:
df_grouped.columns

MultiIndex(levels=[['id', 'user_name'], ['min', 'max', '']],
           labels=[[1, 0, 0], [2, 0, 1]])

In [18]:
df_grouped.head()

user_name                   id                     
                                     min                  max
0      @0001Cecilia  1440785071561064449  1440786056580788227
1    @0071061Mukesh  1438462615609483264  1440710763065208832
2  @00CarlosCabrera  1440746723811282956  1440749888560504837
3           @01kris  1440766565511696389  1440769807733780486
4     @07VinodYadav  1438192604529639424  1440279062388174860

In [21]:
df_grouped = pd.concat([df_grouped.poi_name, df_grouped.id.rename(columns={'min': 'min_id', 'max': 'max_id'})], ignore_index=False, axis=1)


In [25]:
df_grouped.to_csv('df_poi_reply_raw.csv', index=False)

In [ ]:
{
   "ip": "18.118.50.157",
   "port":"8983",
   "covid_keywords": ["quarentena",
  "hospital",
  "covidresources",
  "rt-pcr",
  "वैश्विकमहामारी",
  "oxygen",
  "सुरक्षित रहें",
  "stayhomestaysafe",
  "covid19",
  "quarantine",
  "मास्क",
  "face mask",
  "covidsecondwaveinindia",
  "flattenthecurve",
  "corona virus",
  "wuhan",
  "cierredeemergencia",
  "autoaislamiento",
  "sintomas",
  "covid positive",
  "casos",
  "कोविड मृत्यु",
  "स्वयं चुना एकांत",
  "stay safe",
  "#deltavariant",
  "covid symptoms",
  "sarscov2",
  "covidiots",
  "brote",
  "alcohol en gel",
  "disease",
  "asintomático",
  "टीकाकरण",
  "encierro",
  "covidiot",
  "covidappropriatebehaviour",
  "fever",
  "pandemia de covid-19",
  "wearamask",
  "flatten the curve",
  "oxígeno",
  "desinfectante",
  "super-spreader",
  "ventilador",
  "coronawarriors",
  "quedate en casa",
  "mascaras",
  "mascara facial",
  "trabajar desde casa",
  "संगरोध",
  "immunity",
  "स्वयं संगरोध",
  "डेल्टा संस्करण",
  "mask mandate",
  "health",
  "dogajkidoori",
  "travelban",
  "cilindro de oxígeno",
  "covid",
  "staysafe",
  "variant",
  "yomequedoencasa",
  "doctor",
  "एंटीबॉडी",
  "दूसरी लहर",
  "distancia social",
  "मुखौटा",
  "covid test",
  "अस्पताल",
  "covid deaths",
  "कोविड19",
  "muvariant",
  "susanadistancia",
  "personal protective equipment",
  "remdisivir",
  "quedateencasa",
  "asymptomatic",
  "social distancing",
  "distanciamiento social",
  "cdc",
  "transmission",
  "epidemic",
  "social distance",
  "herd immunity",
  "transmisión",
  "सैनिटाइज़र",
  "indiafightscorona",
  "surgical mask",
  "facemask",
  "desinfectar",
  "वायरस",
  "संक्रमण",
  "symptoms",
  "सामाजिक दूरी",
  "covid cases",
  "ppe",
  "sars",
  "autocuarentena",
  "प्रक्षालक",
  "breakthechain",
  "stayhomesavelives",
  "coronavirusupdates",
  "sanitize",
  "covidinquirynow",
  "कोरोना",
  "workfromhome",
  "outbreak",
  "flu",
  "sanitizer",
  "distanciamientosocial",
  "variante",
  "कोविड 19",
  "कोविड-19",
  "covid pneumonia",
  "कोविड",
  "pandemic",
  "icu",
  "वाइरस",
  "contagios",
  "वेंटिलेटर",
  "washyourhands",
  "n95",
  "stayhome",
  "lavadodemanos",
  "fauci",
  "रोग प्रतिरोधक शक्ति",
  "maskmandate",
  "डेल्टा",
  "कोविड महामारी",
  "third wave",
  "epidemia",
  "fiebre",
  "मौत",
  "travel ban",
  "फ़्लू",
  "muerte",
  "स्वच्छ",
  "washhands",
  "enfermedad",
  "contagio",
  "infección",
  "faceshield",
  "self-quarantine",
  "remdesivir",
  "oxygen cylinder",
  "mypandemicsurvivalplan",
  "कोविड के केस",
  "delta variant",
  "wuhan virus",
  "लक्षण",
  "corona",
  "maskup",
  "gocoronago",
  "death",
  "curfew",
  "socialdistance",
  "second wave",
  "máscara",
  "stayathome",
  "positive",
  "lockdown",
  "propagación en la comunidad",
  "तीसरी लहर",
  "aislamiento",
  "rtpcr",
  "coronavirus",
  "variante delta",
  "distanciasocial",
  "cubrebocas",
  "घर पर रहें",
  "socialdistancing",
  "covidwarriors",
  "प्रकोप",
  "covid-19",
  "stay home",
  "संक्रमित",
  "jantacurfew",
  "cowin",
  "कोरोनावाइरस",
  "virus",
  "distanciamiento",
  "cuarentena",
  "indiafightscovid19",
  "healthcare",
  "natocorona",
  "मास्क पहनें",
  "delta",
  "ऑक्सीजन",
  "wearmask",
  "कोरोनावायरस",
  "ventilator",
  "pneumonia",
  "maskupindia",
  "ppe kit",
  "sars-cov-2",
  "testing",
  "fightagainstcovid19",
  "महामारी",
  "नियंत्रण क्षेत्र",
  "who",
  "mask",
  "pandemia",
  "deltavariant",
  "वैश्विक महामारी",
  "रोग",
  "síntomas",
  "work from home",
  "antibodies",
  "masks",
  "confinamiento",
  "flattening the curve",
  "मुखौटा जनादेश",
  "thirdwave",
  "mascarilla",
  "usacubrebocas",
  "covidemergency",
  "inmunidad",
  "cierre de emergencia",
  "self-isolation",
  "स्वास्थ्य सेवा",
  "सोशल डिस्टन्सिंग",
  "isolation",
  "cases",
  "community spread",
  "unite2fightcorona",
  "oxygencrisis",
  "containment zones",
  "homequarantine",
  "स्पर्शोन्मुख",
  "लॉकडाउन",
  "hospitalización",
  "incubation period"],
   "vaccine_keywords": ["anticuerpos",
  "vaccine mandate",
  "eficacia de la vacuna",
  "vacuna covid",
  "covidvaccine",
  "zycov-d",
  "vaccines",
  "#largestvaccinedrive",
  "vaccination",
  "dosis de vacuna",
  "moderna",
  "campaña de vacunación",
  "vaccineshortage",
  "vacunar",
  "covid vaccine",
  "efectos secundarios de la vacuna",
  "कोविशील्ड",
  "hydroxychloroquine",
  "efficacy",
  "टीके",
  "टीकाकरण",
  "वैक्सीनेशन",
  "shots",
  "covishield",
  "vaccine",
  "antibody",
  "j&j vaccine",
  "booster shot",
  "वैक्सीन पासपोर्ट",
  "covidvaccination",
  "दूसरी खुराक",
  "inyección de refuerzo",
  "astrazeneca",
  "टीकाकरण अभियान",
  "vacunacovid19",
  "johnson & johnson",
  "पहली खुराक",
  "sinopharm",
  "immunity",
  "vaccination drive",
  "inmunización",
  "vaccine dose",
  "we4vaccine",
  "पूर्ण टीकाकरण",
  "vaccine passports",
  "एंटीबॉडी",
  "vacunado",
  "vacunarse",
  "johnson",
  "efecto secundario",
  "astra zeneca",
  "yomevacunoseguro",
  "injection",
  "cdc",
  "वैक्सीन के साइड इफेक्ट",
  "getvaxxed",
  "teeka",
  "टीका",
  "herd immunity",
  "वैक्सीन जनादेश",
  "vaccinepassports",
  "estrategiadevacunación",
  "ivermectin",
  "cansino",
  "vacunas",
  "vaccinehesitancy",
  "sputnik",
  "johnson & johnson’s janssen",
  "unvaccinated",
  "janssen",
  "sputnik v",
  "vacunaton",
  "seconddose",
  "कोवेक्सिन",
  "getvaccinatednow",
  "tikakaran",
  "कोविशिल्ड",
  "खुराक",
  "covaxine",
  "mrna",
  "first dose",
  "वाइरस",
  "booster shots",
  "dosis",
  "side effect",
  "रोग प्रतिरोधक शक्ति",
  "jab",
  "get vaccinated",
  "vaccinessavelives",
  "pinchazo",
  "vaccinesideeffects",
  "vaccinated",
  "कोविड का टीका",
  "खराब असर",
  "vacunación",
  "कोवैक्सिन",
  "tikautsav",
  "efectos secundarios",
  "remdesivir",
  "covid19vaccine",
  "eficacia",
  "anticuerpo",
  "vaccinequity",
  "vaccinesamvaad",
  "फाइजर",
  "vaccinesamvad",
  "covid-19 vaccine",
  "pasaporte de vacuna",
  "largestvaccinationdrive",
  "firstdose",
  "doses",
  "vacuna",
  "la inmunidad de grupo",
  "कोवैक्सीन",
  "vaccine side effects",
  "कोविन",
  "vaccinationdrive",
  "clinical trial",
  "vaccinemandate",
  "segunda dosis",
  "cowin",
  "vaccinate",
  "clinical trials",
  "fully vaccinated",
  "johnson and johnson",
  "primera dosis",
  "largestvaccinedrive",
  "vaccine hesitancy",
  "वैक्सीन",
  "प्रभाव",
  "vacunacion",
  "second dose",
  "sabkovaccinemuftvaccine",
  "लसीकरण",
  "vaccineswork",
  "वैक्\u200dसीन",
  "दुष्प्रभाव",
  "pfizer",
  "vaccine efficacy",
  "टीका लगवाएं",
  "एमआरएनए वैक्सीन",
  "antibodies",
  "getvaccinated",
  "covidshield",
  "booster",
  "टीका_जीत_का",
  "vaccine jab",
  "vaccine passport",
  "vaccinepassport",
  "mrna vaccine",
  "inmunidad",
  "एस्ट्राजेनेका",
  "mandato de vacuna",
  "astrazenca",
  "vacúnate",
  "vacuna para el covid-19",
  "vacunada",
  "side effects",
  "dose",
  "novavax",
  "j&j",
  "covaxin",
  "fullyvaccinated",
  "sputnikv",
  "कोविड टीका",
  "completamente vacunado",
  "novaccinepassports",
  "sinovac"],
   "core":"IRF21P1"
  }

In [169]:
len(df_new)

122055

In [161]:
df_new.head(2)

,country,hashtags,id,mentions,poi_id,poi_name,replied_to_tweet_id,replied_to_user_id,reply_text,text_en,text_es,text_hi,tweet_date,tweet_emoticons,tweet_lang,tweet_text,tweet_urls,verified
0,USA,[#WeCanDoThis],1440345427421786118,NaN,44783853,@HHSGov,0,0,0,COVID 19 vaccine favorite fall activities safe...,NaN,NaN,2021-09-21 16:00:00,NaN,en,"With a COVID-19 vaccine, your favorite fall ac...",[http://vaccines.gov],True
1,USA,"[#COVID19, #vaccine]",1440300231539625986,[@US_FDA],44783853,@HHSGov,0,0,0,Get FDA approved COVID19 vaccine today Find va...,NaN,NaN,2021-09-21 13:00:00,NaN,en,Get your @US_FDA approved #COVID19 #vaccine to...,[http://vaccines.gov],True


In [170]:
df_new.to_csv('input_df.csv', index=False)

In [193]:
# Non POI Covid19 vaccine tweets

df_non_poi = df_new[df_new.poi_id == 0][df_non_poi.replied_to_tweet_id == 0]
def if_vaccine_tweet(text):
    text = text.lower()
    for kw in vaccine_kwds:
        if kw in text:
            return 1
    return 0

df_non_poi['is_vaccine_tweet'] = df_non_poi.tweet_text.apply(if_vaccine_tweet)
print(df_non_poi[df_non_poi.is_vaccine_tweet == 1].id.nunique())

43954


In [180]:
# Reply count

print(df_new[df_new.replied_to_tweet_id != 0].id.nunique())

26681


In [184]:
# Poi Health Reply counts

df_poi = df_new[df_new.poi_id != 0]

def if_covid_tweet(text):
    text = text.lower()
    for kw in vaccine_kwds+covid_kws:
        if kw in text:
            return 1
    return 0

df_poi['is_covid_tweet'] = df_poi.tweet_text.apply(if_covid_tweet)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [186]:
tweet_ids = list(df_poi[df_poi.is_covid_tweet == 1].id.unique())

In [188]:
tweet_ids = list(map(int, tweet_ids))

In [190]:
df_replies = df_new[df_new.replied_to_tweet_id.isin(tweet_ids)]

In [191]:
len(df_replies)

296

In [195]:
d = {
    "ip": "18.118.50.157",
    "port": "8983",
    "covid_keywords": [
        "quarentena",
        "hospital",
        "covidresources",
        "rt-pcr",
        "वैश्विकमहामारी",
        "oxygen",
        "सुरक्षित रहें",
        "stayhomestaysafe",
        "covid19",
        "quarantine",
        "मास्क",
        "face mask",
        "covidsecondwaveinindia",
        "flattenthecurve",
        "corona virus",
        "wuhan",
        "cierredeemergencia",
        "autoaislamiento",
        "sintomas",
        "covid positive",
        "casos",
        "कोविड मृत्यु",
        "स्वयं चुना एकांत",
        "stay safe",
        "#deltavariant",
        "covid symptoms",
        "sarscov2",
        "covidiots",
        "brote",
        "alcohol en gel",
        "disease",
        "asintomático",
        "टीकाकरण",
        "encierro",
        "covidiot",
        "covidappropriatebehaviour",
        "fever",
        "pandemia de covid-19",
        "wearamask",
        "flatten the curve",
        "oxígeno",
        "desinfectante",
        "super-spreader",
        "ventilador",
        "coronawarriors",
        "quedate en casa",
        "mascaras",
        "mascara facial",
        "trabajar desde casa",
        "संगरोध",
        "immunity",
        "स्वयं संगरोध",
        "डेल्टा संस्करण",
        "mask mandate",
        "health",
        "dogajkidoori",
        "travelban",
        "cilindro de oxígeno",
        "covid",
        "staysafe",
        "variant",
        "yomequedoencasa",
        "doctor",
        "एंटीबॉडी",
        "दूसरी लहर",
        "distancia social",
        "मुखौटा",
        "covid test",
        "अस्पताल",
        "covid deaths",
        "कोविड19",
        "muvariant",
        "susanadistancia",
        "personal protective equipment",
        "remdisivir",
        "quedateencasa",
        "asymptomatic",
        "social distancing",
        "distanciamiento social",
        "cdc",
        "transmission",
        "epidemic",
        "social distance",
        "herd immunity",
        "transmisión",
        "सैनिटाइज़र",
        "indiafightscorona",
        "surgical mask",
        "facemask",
        "desinfectar",
        "वायरस",
        "संक्रमण",
        "symptoms",
        "सामाजिक दूरी",
        "covid cases",
        "ppe",
        "sars",
        "autocuarentena",
        "प्रक्षालक",
        "breakthechain",
        "stayhomesavelives",
        "coronavirusupdates",
        "sanitize",
        "covidinquirynow",
        "कोरोना",
        "workfromhome",
        "outbreak",
        "flu",
        "sanitizer",
        "distanciamientosocial",
        "variante",
        "कोविड 19",
        "कोविड-19",
        "covid pneumonia",
        "कोविड",
        "pandemic",
        "icu",
        "वाइरस",
        "contagios",
        "वेंटिलेटर",
        "washyourhands",
        "n95",
        "stayhome",
        "lavadodemanos",
        "fauci",
        "रोग प्रतिरोधक शक्ति",
        "maskmandate",
        "डेल्टा",
        "कोविड महामारी",
        "third wave",
        "epidemia",
        "fiebre",
        "मौत",
        "travel ban",
        "फ़्लू",
        "muerte",
        "स्वच्छ",
        "washhands",
        "enfermedad",
        "contagio",
        "infección",
        "faceshield",
        "self-quarantine",
        "remdesivir",
        "oxygen cylinder",
        "mypandemicsurvivalplan",
        "कोविड के केस",
        "delta variant",
        "wuhan virus",
        "लक्षण",
        "corona",
        "maskup",
        "gocoronago",
        "death",
        "curfew",
        "socialdistance",
        "second wave",
        "máscara",
        "stayathome",
        "positive",
        "lockdown",
        "propagación en la comunidad",
        "तीसरी लहर",
        "aislamiento",
        "rtpcr",
        "coronavirus",
        "variante delta",
        "distanciasocial",
        "cubrebocas",
        "घर पर रहें",
        "socialdistancing",
        "covidwarriors",
        "प्रकोप",
        "covid-19",
        "stay home",
        "संक्रमित",
        "jantacurfew",
        "cowin",
        "कोरोनावाइरस",
        "virus",
        "distanciamiento",
        "cuarentena",
        "indiafightscovid19",
        "healthcare",
        "natocorona",
        "मास्क पहनें",
        "delta",
        "ऑक्सीजन",
        "wearmask",
        "कोरोनावायरस",
        "ventilator",
        "pneumonia",
        "maskupindia",
        "ppe kit",
        "sars-cov-2",
        "testing",
        "fightagainstcovid19",
        "महामारी",
        "नियंत्रण क्षेत्र",
        "who",
        "mask",
        "pandemia",
        "deltavariant",
        "वैश्विक महामारी",
        "रोग",
        "síntomas",
        "work from home",
        "antibodies",
        "masks",
        "confinamiento",
        "flattening the curve",
        "मुखौटा जनादेश",
        "thirdwave",
        "mascarilla",
        "usacubrebocas",
        "covidemergency",
        "inmunidad",
        "cierre de emergencia",
        "self-isolation",
        "स्वास्थ्य सेवा",
        "सोशल डिस्टन्सिंग",
        "isolation",
        "cases",
        "community spread",
        "unite2fightcorona",
        "oxygencrisis",
        "containment zones",
        "homequarantine",
        "स्पर्शोन्मुख",
        "लॉकडाउन",
        "hospitalización",
        "incubation period"
    ],
    "vaccine_keywords": [
        "anticuerpos",
        "vaccine mandate",
        "eficacia de la vacuna",
        "vacuna covid",
        "covidvaccine",
        "zycov-d",
        "vaccines",
        "#largestvaccinedrive",
        "vaccination",
        "dosis de vacuna",
        "moderna",
        "campaña de vacunación",
        "vaccineshortage",
        "vacunar",
        "covid vaccine",
        "efectos secundarios de la vacuna",
        "कोविशील्ड",
        "hydroxychloroquine",
        "efficacy",
        "टीके",
        "टीकाकरण",
        "वैक्सीनेशन",
        "shots",
        "covishield",
        "vaccine",
        "antibody",
        "j&j vaccine",
        "booster shot",
        "वैक्सीन पासपोर्ट",
        "covidvaccination",
        "दूसरी खुराक",
        "inyección de refuerzo",
        "astrazeneca",
        "टीकाकरण अभियान",
        "vacunacovid19",
        "johnson & johnson",
        "पहली खुराक",
        "sinopharm",
        "immunity",
        "vaccination drive",
        "inmunización",
        "vaccine dose",
        "we4vaccine",
        "पूर्ण टीकाकरण",
        "vaccine passports",
        "एंटीबॉडी",
        "vacunado",
        "vacunarse",
        "johnson",
        "efecto secundario",
        "astra zeneca",
        "yomevacunoseguro",
        "injection",
        "cdc",
        "वैक्सीन के साइड इफेक्ट",
        "getvaxxed",
        "teeka",
        "टीका",
        "herd immunity",
        "वैक्सीन जनादेश",
        "vaccinepassports",
        "estrategiadevacunación",
        "ivermectin",
        "cansino",
        "vacunas",
        "vaccinehesitancy",
        "sputnik",
        "johnson & johnson’s janssen",
        "unvaccinated",
        "janssen",
        "sputnik v",
        "vacunaton",
        "seconddose",
        "कोवेक्सिन",
        "getvaccinatednow",
        "tikakaran",
        "कोविशिल्ड",
        "खुराक",
        "covaxine",
        "mrna",
        "first dose",
        "वाइरस",
        "booster shots",
        "dosis",
        "side effect",
        "रोग प्रतिरोधक शक्ति",
        "jab",
        "get vaccinated",
        "vaccinessavelives",
        "pinchazo",
        "vaccinesideeffects",
        "vaccinated",
        "कोविड का टीका",
        "खराब असर",
        "vacunación",
        "कोवैक्सिन",
        "tikautsav",
        "efectos secundarios",
        "remdesivir",
        "covid19vaccine",
        "eficacia",
        "anticuerpo",
        "vaccinequity",
        "vaccinesamvaad",
        "फाइजर",
        "vaccinesamvad",
        "covid-19 vaccine",
        "pasaporte de vacuna",
        "largestvaccinationdrive",
        "firstdose",
        "doses",
        "vacuna",
        "la inmunidad de grupo",
        "कोवैक्सीन",
        "vaccine side effects",
        "कोविन",
        "vaccinationdrive",
        "clinical trial",
        "vaccinemandate",
        "segunda dosis",
        "cowin",
        "vaccinate",
        "clinical trials",
        "fully vaccinated",
        "johnson and johnson",
        "primera dosis",
        "largestvaccinedrive",
        "vaccine hesitancy",
        "वैक्सीन",
        "प्रभाव",
        "vacunacion",
        "second dose",
        "sabkovaccinemuftvaccine",
        "लसीकरण",
        "vaccineswork",
        "दुष्प्रभाव",
        "pfizer",
        "vaccine efficacy",
        "टीका लगवाएं",
        "एमआरएनए वैक्सीन",
        "antibodies",
        "getvaccinated",
        "covidshield",
        "booster",
        "टीका_जीत_का",
        "vaccine jab",
        "vaccine passport",
        "vaccinepassport",
        "mrna vaccine",
        "inmunidad",
        "एस्ट्राजेनेका",
        "mandato de vacuna",
        "astrazenca",
        "vacúnate",
        "vacuna para el covid-19",
        "vacunada",
        "side effects",
        "dose",
        "novavax",
        "j&j",
        "covaxin",
        "fullyvaccinated",
        "sputnikv",
        "कोविड टीका",
        "completamente vacunado",
        "novaccinepassports",
        "sinovac"
    ],
    "core": "IRF21P1_demo"
}

In [196]:
with open('project1_index_details.pickle', 'wb') as f:
    pickle.dump(d, f, protocol=pickle.HIGHEST_PROTOCOL)

In [201]:
import requests
import time

payload = pickle.load(open("project1_index_details.pickle", "rb"))
t1 = time.time()
res = requests.post("http://54.89.143.10:9998/grade_index", json=payload, timeout=600)
t2 = time.time()
res = res.json()
print(res)
print(t2-t1)

{'Report': {'Country Volumes': {'comments': '5000 distinct tweets found for each of USA, Mexico and India.', 'score': 1}, 'Data Sanity': {'comments': 'All fields present. All field types match. Multivalue flag for all fields match. ', 'score': 0.5}, 'Data Sanity Live Check': {'comments': 'Passed random matching of N actual tweets from Twitter.', 'score': 0.5}, 'Language Volumes': {'comments': '5000 distinct tweets found for each of en, es and hi.', 'score': 1}, 'NON POI health related Tweet Volume Count': {'comments': '17593 NON POI tweets found, which contain Covid-19 vaccine related keywords.', 'score': 0.5413}, 'Non POI Reply Count': {'comments': '1307 Non POIs have >= 1 reply to other tweets. ', 'score': 0.8713}, 'POI Count': {'comments': '25 POIs found, which is >= 15', 'score': 0.5}, 'POI Health Reply Count': {'comments': '18 POI health related tweets have >= 10 replies.', 'score': 0.12}, 'POI Tweet Volume Count': {'comments': '25 POIs have >= 500 tweets.', 'score': 0.5}, 'POI he